LDA алгоритм хорошо справляется с большим массивом текстов


In [2]:
import numpy as np
import json
import glob

#gensim
import gensim 
import gensim. corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords 

#visualisation
import pyLDAvis
import pyLDAvis.gensim

stopwords = stopwords.words("russian")

In [3]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)


In [4]:
data = load_data('/Users/gleb/Desktop/internet_lab_project/docs/clean_docs/lst.json')['text']
data[0][0:90]

'Одобрена решением Государственной комиссии информатизации Государственном комитете Российс'

In [5]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    nlp = spacy.load('ru_core_news_lg', disable=['parser', 'ner']) 
    texts_out = []
    for t in texts:
        doc = nlp(t)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = ' '.join(new_text)        
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print(lemmatized_texts[0][0:100])

одобрить решение государственный комиссия информатизация государственный комитет российский связь ин


In [6]:
#токенизация
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print(data_words[0][0:20])

['одобрить', 'решение', 'государственныи', 'комиссия', 'информатизация', 'государственныи', 'комитет', 'россиискии', 'связь', 'информатизация', 'настоящии', 'время', 'осознать', 'предпосылка', 'реальныи', 'путь', 'формирование', 'развитие', 'информационныи', 'общество']


In [29]:
#BIGRAMS AND TRIGRAMS

bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=20)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=20)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

# print (data_bigrams_trigrams[0][0:20])
for i in list(data_bigrams_trigrams)[0]:
    if '_' in i:
        print(i)



# keywords to file
with open ("/Users/gleb/Desktop/internet_lab_project/code/k-w_extractions/TF-IDF/results/bigrams.txt ", "w", encoding="utf-8") as d:
    for j in list(data_bigrams_trigrams):
        for i in j:
            if '_' in i:    
                d.write(i)
                d.write('\n')
        d.write("------------")
        d.write('\n')

гражданскии_общество
вычислительныи_техника
продукт_услуга
образ_жизнь
образ_жизнь
образ_жизнь
продукт_услуга
образ_жизнь
последнии_год
объем_продажа
вычислительныи_техника
млрд_доллар
вычислительныи_техника
млрд_доллар
продукт_услуга
сеть_связь
политическии_экономическии
политическии_экономическии
процесс_переход
повышение_качество
процесс_переход
политическии_экономическии
продукт_услуга
процесс_переход
процесс_переход
государственныи_политика
мировои_рынок
продукт_услуга
процесс_переход
процесс_переход
процесс_переход
орган_власть
процесс_переход
продукт_услуга
политическии_экономическии
процесс_переход
политическии_экономическии
орган_власть
политическии_экономическии
продукт_услуга
продукт_услуга
высокии_уровень
темп_рост
продукт_услуга
высокии_уровень
продукт_услуга
высокии_уровень
продукт_услуга
образ_жизнь
высокии_уровень
повышение_качество
гражданскии_общество
политическии_экономическии
образ_жизнь
процесс_переход
высокии_уровень
продукт_услуга
государственныи_политика
продукт

In [17]:
# 2 вариант

# tf-idf removal

from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
# print(corpus)

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] 
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]
    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

In [13]:
#  1 вариант

# # Bag-of-words
# # создаем словарь
# # corpora - удобная библиотека для хранения пары ключ-значение для корпусов текстов огромных размеров
# id2word = corpora.Dictionary(data_words)

# corpus = []
# for text in data_words:
#     new = id2word.doc2bow(text)
#     corpus.append(new)
    
# print('Первое число это индекс слова, второе - кол-во повторений этого слова')
# print(corpus[0][0:20])

# word = id2word[[0][:1][0]]
# word

In [30]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=15,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha="auto")

<h2>VIZ</h2>

In [31]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model,corpus, id2word, mds='mmds', R=30)
vis

/Users/gleb/Desktop/internet_lab_project/code/venv/lib/python3.12/site-packages/joblib/_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
/Users/gleb/Desktop/internet_lab_project/code/venv/lib/python3.12/site-packages/joblib/_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
/Users/gleb/Desktop/internet_lab_project/code/venv/lib/python3.12/site-packages/joblib/_utils.py:39: DeprecationWarning: Attribute n is deprecated and will be removed in Python 3.14; use value instead
  return node.n
/Users/gleb/Desktop/internet_lab_project/code/venv/lib/python3.12/site-packages/joblib/_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
/Users/gleb/Desktop/internet_lab_projec

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.293895 -0.233997       1        1  54.301827
6      0.098157 -0.302934       2        1  21.024260
4     -0.278542  0.173191       3        1  17.948295
12     0.213373  0.002362       4        1   6.698225
5      0.024783  0.031834       5        1   0.002884
1      0.023623  0.033119       6        1   0.002463
7      0.023133  0.032531       7        1   0.002462
14     0.023307  0.032141       8        1   0.002461
10     0.023540  0.032823       9        1   0.002461
13     0.023846  0.033293      10        1   0.002461
8      0.023882  0.032986      11        1   0.002460
0      0.023620  0.032872      12        1   0.002440
11     0.023621  0.032958      13        1   0.002439
2      0.023803  0.033355      14        1   0.002432
9      0.023750  0.033465      15        1   0.002431, topic_info=                                         Term        Freq       Total  \
1072                                 компания  109.000000  109.000000   
821                                специалист   36.000000   36.000000   
654                                   процент   32.000000   32.000000   
1030                                  включая   31.000000   31.000000   
988                                     часть   26.000000   26.000000   
...                                       ...         ...         ...   
1472                                  крупныи    0.000041   24.963566   
1165  федеральныи_орган_исполнительныи_власть    0.000041    6.677194   
2118                                     лицо    0.000041    5.873297   
1089                             оборудование    0.000041    9.335066   
1677                                   подход    0.000041   11.734145   

     Category  logprob  loglift  
1072  Default  30.0000  30.0000  
821   Default  29.0000  29.0000  
654   Default  28.0000  28.0000  
1030  Default  27.0000  27.0000  
988   Default  26.0000  26.0000  
...       ...      ...      ...  
1472  Topic15  -7.6603  -2.6865  
1165  Topic15  -7.6738  -1.3813  
2118  Topic15  -7.6777  -1.2569  
1089  Topic15  -7.6732  -1.7158  
1677  Topic15  -7.6747  -1.9460  

[901 rows x 6 columns], token_table=      Topic      Freq                   Term
term                                        
1190      3  0.939941                 анализ
1023      4  1.223408               архивныи
13        2  0.924135                   база
2052      3  0.934815             безопасныи
1025      4  0.972527           библиотечныи
...     ...       ...                    ...
988       1  0.984917                  часть
1000      2  0.976911                широкии
1170      1  0.852379  широкополосныи_доступ
1170      4  0.142063  широкополосныи_доступ
2265      3  0.987953      электронныи_форме

[224 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 7, 5, 13, 6, 2, 8, 15, 11, 14, 9, 1, 12, 3, 10])